**Actual Machine Learning**

In [2]:
from pathlib import Path
import Augmentation
import torchaudio
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torchvision import datasets
import numpy as np
from torch.utils.data import Dataset, DataLoader
from datetime import datetime
import os
import mlmodel

audio_paths = Augmentation.getAudioPaths('./data/')

audio_train_paths, audio_val_paths = audio_paths[:int(
    0.8 * len(audio_paths))], audio_paths[int(0.8 * len(audio_paths)):]


audio_train_dataset = Augmentation.AudioDataset(
    audio_train_paths,
    transformList=[
        torchaudio.transforms.TimeMasking(time_mask_param=80),
        torchaudio.transforms.FrequencyMasking(freq_mask_param=80),
    ])

audio_val_dataset = Augmentation.AudioDataset(audio_val_paths)

train_dataloader = torch.utils.data.DataLoader(audio_train_dataset,
                                               batch_size=4,
                                               num_workers=0,
                                               shuffle=True)

val_dataloader = torch.utils.data.DataLoader(audio_val_dataset,
                                              batch_size=4,
                                              num_workers=0,
                                              shuffle=True,
                                             )

model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18')

model.conv1 = nn.Conv2d(1,
                        64,
                        kernel_size=(7, 7),
                        stride=(2, 2),
                        padding=(3, 3),
                        bias=False)

model.fc = nn.Linear(in_features=512, out_features=10, bias=True)


title = datetime.now().strftime("%Y-%m-%d,%H-%M-%S")
title = False

mymodel = mlmodel.mlmodel(model, train_dataloader, val_dataloader, title)

ModuleNotFoundError: No module named 'mlmodel'

In [ ]:
# mymodel.write_tb_graph(train_dataloader)

In [ ]:
cost = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def interateModel(epochs):
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}\n-------------------------------')
        train_loss, train_accuracy = mymodel.train(cost, optimizer)
        val_loss, val_accuracy = mymodel.val(cost)
        tune.report(mean_accuracy=val_accuracy)
        print(f'Training | Loss: {train_loss} Accuracy: {train_accuracy}%')
        print(f'Validating  | Loss: {val_loss} Accuracy: {val_accuracy}% \n')
        mymodel.tensorBoardLogging(train_loss, train_accuracy, val_loss,
                                   val_accuracy, epoch)
        print('Done!')


interateModel(10)

In [ ]:
# model.load_state_dict(
#     torch.load("./model/model_t4,f4,tf4.pt", map_location=device))

In [ ]:
torch.save(mymodel.getModel().state_dict(), f"./model/model_t4,f4,tf4.pt")